In [1]:
#Importing Data from the CSV file
import string

f_train = open("training.1600000.processed.noemoticon.csv", "r", encoding="ISO-8859-1")
X_train = f_train.readlines()
# for i in range(5):
#     print(X_train[i])
f_test = open("testdata.manual.2009.06.14.csv", "r")
X_test = f_test.readlines()
# for i in range(10):
#     print(X_test[i])

#Creating the List with just the tweets and finding the number of positive and negative classes. (TRAINING) 
class_0=class_4=0
tweets_train =[]
for x in X_train:
    a = x.split('","') #could also use '","' as the splitter - compare with "0 for class and append a[-1] in list"
    if (a[0] == '"0'): class_0+=1
    else: class_4+=1
    tweets_train.append('%s"|"%s' % (a[0],a[-1]))

#Creating the List with just the tweets and finding the number of positive and negative classes. (TRAINING) 
tweets_test =[]
for x in X_test:
    a = x.split('","')
    tweets_test.append('%s"|"%s' % (a[0], a[-1]))

    
print("The length of the training set is = ", len(X_train))
print("The number of +ve classes (label=4) =", class_4)
print("The number of -ve classes (label=0) =", class_0)
print("The length of the test set is = ", len(X_test))

f_test.close()
f_train.close()

The length of the training set is =  1600000
The number of +ve classes (label=4) = 800000
The number of -ve classes (label=0) = 800000
The length of the test set is =  359


In [2]:
!rm -rf tweet_test.txt
!rm -rf tweet_train.txt
f = open("tweet_train.txt", "w")
for x in tweets_train:
    #x = x.split("|")
    f.write(x)
f.close()

f = open("tweet_test.txt", "w")
for x in tweets_test:
    #x = x.split("|")[1]
    f.write(x)
f.close()

In [3]:
dictionary = {}
dict_0 = {}
dict_4 = {}
n = []
for x in tweets_train:
    x = x.rstrip("\n\r")
    x = x.split("|")
    twt = x[1].translate(str.maketrans('','',string.punctuation)).lower()
    words = twt.split() # if I use split(" "), it takes more than one white space as a word
    n.append(len(words))
    for w in words:
#Global Vocabulary
        if w in dictionary:
            dictionary[w]+=1
        else:
            dictionary[w]=1
#build Vocabulary for class 0 (-ve class)
        if (x[0] == '"0"'):
            if w in dict_0:
                dict_0[w]+=1
            else:
                dict_0[w]=1
#build Vocabulary for class 4 (+ve class)
        else:
            if w in dict_4:
                dict_4[w]+=1
            else:
                dict_4[w]=1
        


In [48]:
m= len(X_train)
phi_0 = class_0/m
phi_4 = class_4/m
v = len(dictionary)
print(v)
n_0 = sum(n[0:class_0])
n_4 = sum(n[class_0:len(n)])
print(n_0,n_4)

849585
10760560 10085157


In [55]:
import math
theta_0 = {}
theta_4 = {}
n_0 = sum(n[0:class_0])
n_4 = sum(n[class_0:len(n)])
c=1
for word in dictionary.keys():
        if word in dict_0:
            theta_0[word] = ((dict_0[word]+c)/(n_0 + v*c))
        else:
            theta_0[word] = ((c) / (n_0 + v*c))
        if word in dict_4:
            theta_4[word] = ((dict_4[word]+c)/(n_4 + v*c))
        else:
            theta_4[word] = ((c)/(n_4 + v*c))

In [13]:
print(len(theta_0))
print(len(theta_4))

849585
849585
i 0.026390288860953463


In [40]:
acc = []
for x in tweets_test:
    test_class0=test_class4=0
    p_x = 1
    x = x.rstrip("\n\r")
    x = x.split("|")
    twt = x[1].translate(str.maketrans('','',string.punctuation)).lower()
    words = twt.split() # if I use split(" "), it takes more than one white space as a word
    for w in words:
        if w in theta_0: test_class0 += math.log(theta_0[w])
        else: test_class0 += math.log(1)
        if w in theta_4: test_class4 += math.log(theta_4[w])
        else: test_class4 += math.log(1)
    test_class0 += math.log(phi_0)
    test_class4 += math.log(phi_4)
    #test_class0 /= (test_class0+test_class4)
    #test_class4 /= (test_class0+test_class4)
    print(test_class0, "\t", test_class4)

-132.56120581946558 	 -134.06629683477632
-82.84697193462995 	 -80.19689216859871
-76.72305387353266 	 -76.31095169389393
-178.19918777764815 	 -178.35040949786023
-78.18888254153227 	 -80.09205321715493
-90.34642512050507 	 -89.12298662805831
-106.67618191008354 	 -111.83331221553098
-43.97495160618547 	 -42.37265358018858
-17.055467469537025 	 -15.397626902808213
-82.94771603310822 	 -81.01659788479004
-190.41725847508602 	 -191.44326393228158
-150.3822041845464 	 -147.94641109802487
-84.1474273688 	 -83.57614706767102
-136.68236783012517 	 -137.9944588940632
-161.9196661601439 	 -162.8158559292944
-206.0222469661956 	 -211.56717296222374
-21.262075694665324 	 -21.054384722434794
-94.91883734500617 	 -96.0077511984794
-29.70602370401454 	 -30.22482364612435
-116.4466735004903 	 -114.43996401771643
-57.070506715310884 	 -55.329391994622064
-100.31505713197107 	 -101.2134491531305
-116.60427024861677 	 -118.27630556495969
-163.06254782820682 	 -166.13182792591732
-64.5609778292296 	 -6

In [59]:
print(dict_4['cool'])
print(dict_0['cool'])
print(math.log(theta_4['fantastic']))
print(math.log(theta_0['fantastic']))
print(math.log(theta_4['cool']))
print(math.log(theta_0['cool']))

10228
3819
-8.8962372334307
-10.364756509428137
-6.9744735153933215
-8.019384141228883
